# B+Tree

In [1]:
# El módulo abc obliga a las clases derivadas a implemtar
# un método particular utilizando @abstractmethod
from abc import ABCMeta, abstractmethod

# Permite trabajar mejor con listas, insertar conservando el orden
import bisect
from math import floor as floor


In [2]:
floor(5.25)

5

In [3]:
class Node(metaclass=ABCMeta):  
    
    def __init__(self):
        self.num = 0
        self.keys = []
        
    @abstractmethod
    def getLoc(self, key):
        """Return null si no split. Si no retorna la info del split."""
        pass
    
    @abstractmethod
    def insert(self, key,value): pass
    
    @abstractmethod
    def dump(self): pass

In [4]:
class Split:
    
    # TODO: Refactor Split
    
    def __init__(self, key, node_leaf, node_right):
        self.key = key
        self.left = node_leaf
        self.right = node_right
    

In [5]:
class LNode(Node):
    
    def __init__(self):
        super(LNode, self).__init__();
        
        self.values = []
    
    def getLoc(self, key):
        """Busqueda lineal"""
        # for i, key_i in enumerate(self.keys):
        #    if key_i >= key:
        #        return i
        # Si no se encontró una posición retorna la posición final
        # return len(self.keys)
        return bisect.bisect_left(self.keys, key)

            
    def insert(self, key, value):
        i = self.getLoc(key)
        
        if self.num == M:
            # Nodo llego, se debe dividir
            mid = floor((M + 1) / 2)
            sNum = self.num - mid
            
            # Creamos un nodo Hermano, y a este le asignamos la mitad de los elementos
            sibling = LNode()
            sibling.num = sNum
            print("mid: {}".format(mid))
            sibling.keys = self.keys[mid:]
            sibling.values = self.values[mid:]
            
            self.keys = self.keys[:mid]
            self.values = self.values[:mid]
            self.num = mid
            
            if i < mid:
                #self.insertNonfull(key, value, i)
                self.insertNonFull(key, value, i)
            else:
                # nuevo valor va a la derecha
                # sibling.insertNonfull(key, value, i-mid)
                sibling.insertNonFull(key, value, i-mid)
                            
            # notificar al nodo padre
            result = Split(sibling.keys[0], self, sibling)
            
            return result
        
        else:
            self.insertNonFull(key, value, i)
            return None 
        
    def insertNonFull(self, key, value, i):
        self.keys.insert(i, key)
        self.values.insert(i, value)
        self.num += 1          
    
    def dump(self):
        print("LNode h==0")
        for key in self.keys:
            print(key)

In [6]:
class INode(Node):
    
    def __init__(self):
        super(INode, self).__init__();
                
        self.children = [None] * N
    
    def getLoc(self, key):
        # for i, key_i in enumerate(self.keys):
        #    if key_i >= key:
        #        return i
        # Si no se encontró una posición retorna la posición final
        # return len(self.keys)
        return bisect.bisect_left(self.keys, key)
            
    def insert(self, key, value):
        if self.num == N:
            # Dividir
            mid = floor((N + 1) / 2)
            sNum = self.num - mid
            
            sibling = INode()
            sibling.num = sNum
            
            sibling.keys = self.keys[mid:]
            sibling.children = self.children[mid:]
            
            self.keys = self.keys[:mid]
            self.children = self.children[:mid]
            self.num = mid - 1 #
            
            result = Split(self.keys[mid - 1], self, sibling)
            
            # insertar en el lado apropiado            
            if key <= result.key: # menor que cero
                self.insertNonFull(key, value)
            else:
                sibling.insertNonFull(key, value)
            
            return result
        
        else:
            self.insertNonFull(key, value)
            return None
        
    def insertNonFull(self, key, value):
        i = self.getLoc(key)
        print("i:{}".format(i))
        print(self.children)
        result = self.children[i].insert(key, value)
        
        if result is not None:
            # Caso contrario no se debe modicar nada
            
            if i == self.num:
                # Insertamos a la derecha
                
                self.keys[i] = result.key
                self.children[i] = result.left
                self.children[i+1] = result.right
                self.num += 1
                
            else:
                self.children.insert(i, result.left)
                self.children.insert(i + 1, result.right)
                self.keys.insert(i, result.key)
                self.num += 1;              
        
    
    def dump(self):
        print("INode h==0")
        for i, key, child in zip(self.keys, self.children):
                child.dump()                
                print("> {key}".format(key))            
                
        self.children[self.num].dump();       

In [7]:
class BTree:
    
    def __init__(self, degree):
        self.max_leafs = degree - 1
        self.max_inner_nodes = degree
        
        M = self.max_leafs
        N = self.max_inner_nodes
        
        self.root = LNode()
    
    def insert(self, key, value):
        print("Insertar [{}]={}".format(key, value))
        result = self.root.insert(key, value)
        
        if result is not None:
            print(result.key)
            # Se dividió la raiz
            # Se crea una nueva raiz
            _root = INode()
            _root.num = 1
            #_root.keys[0] = result.key
            #_root.keys.append(result.key)
            _root.keys.insert(0, result.key)
            #_root.children[0] = result.left
            #_root.children[1] = result.right
            
            _root.children.insert(0, result.left)
            
            _root.children.insert(1, result.right)
            
            #_root.children.append(result.left)
            #_root.children.append(result.right)
            self.root = _root
    
    def find(self, key):
        node = self.root
        
        while isinstance(node, INode):
            inner = node
            idx = inner.getLoc(key)
            node = inner.children[idx]
        
        # Estamos en la hoja
        idx = node.getLoc(key)
        
        if idx < node.num and node.keys[idx] == key:
            return node.values[idx]
        else:
            return None
    
    def show(self):
        node = self.root
        
        print(node.children)
    
    def dump(self):
        self.root.dump()
    

In [8]:
tree = BTree(4)

In [9]:
# Maximo número de hojas
M = 3;

# Maximo número de nodos en nodo intermedio
N = 4;

In [10]:
print(tree.find(5))

None


In [11]:
tree.insert(1, "1111")
tree.insert(2, "2222")
tree.insert(3, "3333")
tree.insert(4, "4444")


Insertar [1]=1111
Insertar [2]=2222
Insertar [3]=3333
Insertar [4]=4444
mid: 2
3


In [12]:
print(tree.find(4))

4444


In [13]:
tree.show()

[<__main__.LNode object at 0x000001D4B66E1278>, <__main__.LNode object at 0x000001D4B66EB320>, None, None, None, None]


In [17]:
tree.insert(1, "1111")
tree.insert(2, "2222")
tree.insert(3, "3333")
tree.insert(4, "4444")


Insertar [1]=1111
i:0
[<__main__.LNode object at 0x000001D4B66E1278>, <__main__.LNode object at 0x000001D4B66EBB38>, <__main__.LNode object at 0x000001D4B66E1278>, <__main__.LNode object at 0x000001D4B66EB320>, None, None, None, None]
mid: 2
Insertar [2]=2222
i:0
[<__main__.LNode object at 0x000001D4B66E1278>, <__main__.LNode object at 0x000001D4B66EBFD0>, <__main__.LNode object at 0x000001D4B66E1278>, <__main__.LNode object at 0x000001D4B66EBB38>, <__main__.LNode object at 0x000001D4B66E1278>, <__main__.LNode object at 0x000001D4B66EB320>, None, None, None, None]
mid: 2
Insertar [3]=3333
i:1
[<__main__.LNode object at 0x000001D4B66E1278>, <__main__.LNode object at 0x000001D4B66EBDA0>]
4
Insertar [4]=4444
i:0
[<__main__.INode object at 0x000001D4B66EB6A0>, <__main__.INode object at 0x000001D4B66EBE10>, None, None, None, None]
i:1
[<__main__.LNode object at 0x000001D4B66E1278>, <__main__.LNode object at 0x000001D4B66EBDA0>]
mid: 2


AttributeError: 'INode' object has no attribute 'childre'

In [15]:
print(M)

3


In [16]:
print(N)

4
